In [21]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm

In [22]:
#Loading data (here we use the data we simulated in our earlier script)

#Locate my folder -insert your own folder here (remember Python dont understand spaces so use underscore '_')
my_folder='C:/Users/xxx'
x_val = genfromtxt(my_folder+'ARCH1sim.csv', delimiter=',') #loading the csv data and storing it as a np array

In [23]:
def ARCH11_filter(omega, alpha,x_values): #function to find sigma^2 from observations
    T=len(x_values)
    
    #Define the series sigma^2_t (sigma2) as empty variables we assign values to later. 
    sigma2=np.empty(T)
    
    sigma2[0]=1
    
    for t in range(1,T):
        sigma2[t]=omega + alpha*x_values[t-1]**2
    
    return sigma2

def ARCH11_likelihood(omega,alpha):
    
    sigma2=ARCH11_filter(omega, alpha, x_val)
    
    LogL=-np.sum(-np.log(sigma2)-x_val**2/sigma2) #Constants have been removed from the expression
    #Note we take the expression and multiply with (-1) as we are going to minimize the function in the optimization
    
    return LogL

def optim(alpha): #defining an optimization function that only can change on alpha
    omega,alp=([1,alpha]) #Omega fixed to 1
    return ARCH11_likelihood(omega, alp)

In [24]:
alpha0=np.array([0.25]) #initial guesses
res=opt.minimize(optim, alpha0, bounds=((0,None),))

In [26]:
#Now computing Standard errors
v_hessian=res.hess_inv.todense() #retrieves the negative inverse hessian matrix (note we have minimized the negative log likelihood function)
se_hessian=np.sqrt(np.diagonal(v_hessian))

In [29]:
alpha_hat=res.x[0] #res.x gives the parameter estimate(s)

In [46]:
print('Estimates are given by with std. errors in ():')
print('alpha_hat= '+str(alpha_hat)+' (' +str(se_hessian[0])+')'+' ' ' -> T-val=' +str(alpha_hat/se_hessian[0])+'' )
print('Maximized log-likelihood (without constants) ' +str(ARCH11_likelihood(1,alpha_hat)))


Estimates are given by with std. errors in ():
alpha_hat= 0.5132359781625773 (0.03956991017217168)  -> T-val=12.970359951019566
Maximized log-likelihood (without constants) 1455.1838643511683
